In [1]:
from Crypto.PublicKey import RSA
import socket
import MCipher

In [ ]:
# Key generate
prikey = RSA.generate(1024)
pubkey = prikey.publickey()

priFile = open("./serverpriKey.pem" , "wb+")
priFile.write(prikey.exportKey("PEM"))
priFile.close()

pubFile = open("./serverpubKey.pem" , "wb+")
pubFile.write(pubkey.exportKey("PEM"))
pubFile.close()

In [ ]:
def server_program() :
    host = '127.0.0.1'
    port = 5461
    
    key = 'thisisbadkeyokeythisisbadkeyokey'
    iv = 'ivisintialvector'
    
    server_socket = socket.socket()
    server_socket.bind((host, port))
    server_socket.listen(2)
    conn, address = server_socket.accept()
    
    client_pubkey = MCipher.readPEM("clientpubKey.pem")
    key_encrypt = MCipher.RSAEncrypt(client_pubkey , key)[0]

    conn.send(key_encrypt)
    
    print(conn.recv(1024).decode())
    
    conn.send(iv.encode())
    print(conn.recv(1024).decode())
    
    print("Connection from :" + str(address))
    AES = MCipher.setAES(key , iv)
    while True :
        rdata = conn.recv(1024)
        if not rdata :
            break
        d_data = MCipher.AES_Decrypt(AES,rdata)
        data, hash_data = MCipher.separateHashBlcok(d_data)
        
        if MCipher.integrityCheck(data , hash_data) :
            print("무결성 만족")
            print("revc complete.....")
            print("Recieved from user2 :" + str(data))
            
            data = input('->')
            hash_block = MCipher.makeHashBlock(data) 
            hash_block_en = MCipher.AES_Encrypt(AES,hash_block)
            conn.send(hash_block_en)
        
        else :
            print("무결성 위배")
            break
       
        
    conn.close()
    
if __name__ == '__main__' :
    server_program()
    

key exchange Success
iv exchange Success
Connection from :('127.0.0.1', 49963)
무결성 만족
revc complete.....
Recieved from user2 :sdgsdg
->bfbbfbfb
무결성 만족
revc complete.....
Recieved from user2 :sdgsdg
